In [ ]:
#Imports
import os
import numpy as np
import header_maker as hm
import matplotlib.pyplot as plt

In [ ]:
#File paths
base_dir = '/home/stba7609/NII_nebular_phase/'
data_dir = base_dir + 'data_creation/woosley_data/'

In [ ]:
#The cell below contains helper function for reading in the woosley data. They require no changes.
def name_to_number(string): #Function to translate a header name to the corresponding column number
    '''Convert the string to the corresponding
    column index for the woosley data'''
    
    try:
        location = np.where(header_array == string)[0][0]
    except:
        print(string)
        print(header_array)
        sys.exit()
    
    return location    

def skip(skipcols = {}, n = 0): #A handy function for reading in the correct columns from Woosley files
    skipper = sorted(set(range(n))-skipcols)
    return skipper

def get_isotopes(el, exclude = []): #Get all isotopes (their column numbers) of a header name. Can exclude specific atoms
    
    isotopes = []
    
    #Get the alphabetical part of this isotope
    #============================================================================================
    atom = header_array[el]
    atom_letters = ''
    j, letters = 0, True
    while letters == True:
        if atom[j].isalpha() == True: 
            atom_letters += atom[j]
            j += 1
        elif atom[j].isalpha() == False:
            letters = False
            
    #Compare all headers' alphabetic parts with this atom_letters
    #============================================================================================        
    for i in range(len(header_array)):
        header = header_array[i]
        
        if header[:len(atom_letters)] == atom_letters and header[len(atom_letters)].isalpha() == False:
            if header in exclude:
                pass
            else:
                isotopes.append(i)
            
    return isotopes

In [ ]:
#Read in the data and divide in zones

#First define some things
Msun = 1.989*10**33 #g

n_zone = 0
mass = 1
radius = 2
velocity = 3

vmin = 50 #kms
Mej_frac = 0.99 #Cut off the fastest 1% of ejecta

element_names = ['he4', 'c12', 'n14', 'o16', 'ne20', 'mg24', 'si28', 's32', 'ni56']
zone_names = ['Fe/He', 'Si/S', 'O/Si/S', 'O/Ne/Mg', 'O/C', 'He/C', 'He/N']
cap_names = ['He', 'C', 'N', 'O', 'Ne', 'Mg', 'Si', 'S', '56Ni']

#Read in the raw data

temp = open(data_dir + 'he4.0.1p-1d')

header_array = hm.main(temp) #Get the header of the file
cols_to_use = skip({10, 11}, len(header_array)-1)

#Using the header, translate the element names to column numbers
elements_to_use = [] 
for i in range(len(element_names)):
    elements_to_use.append(name_to_number(element_names[i]))

#Now truly read in the data
data = np.loadtxt(temp, skiprows = 6, usecols = cols_to_use)

temp.close()

#============================================================================================

#Determine the zone boundaries
#============================================================================================
fe_he = np.where( data[:, velocity]/(10**5) > vmin )[0][0] #Prevent any fallback material
si_s = np.where( data[fe_he:, name_to_number('he4')] < 10**-2)[0][0] + fe_he
o_si_s = np.where( data[si_s:, name_to_number('o16')] > 10**-2)[0][0] + si_s
o_ne_mg = np.where( data[o_si_s:, name_to_number('ne20')] > data[o_si_s:, name_to_number('si28')])[0][0] + o_si_s
o_c = np.where( data[o_ne_mg:, name_to_number('c12')] > data[o_ne_mg:, name_to_number('ne20')])[0][0] + o_ne_mg
he_c = np.where( data[o_c:, name_to_number('he4')] > data[o_c:, name_to_number('o16')])[0][0] + o_c
he_n = np.where( data[he_c:, name_to_number('n14')] > data[he_c:, name_to_number('c12')])[0][0] + he_c

zones = [fe_he, si_s, o_si_s, o_ne_mg, o_c, he_c, he_n]

#Cut off all layers with velocity greater than vmax
outer_layer = np.where(np.cumsum(data[:, mass])/np.sum(data[:, mass]) > Mej_frac)[0][0]
if outer_layer <= zones[-1]:
    pass
else:
    zones.append(outer_layer)

In [ ]:
#Plot mass versus mass fraction before and after explosion
fig, ax = plt.subplots(1, figsize = (6.5, 3.5))

cum_mass = np.cumsum(data[:, mass])/Msun

#Plot the individual elements
for i in range(len(elements_to_use)):
    el = elements_to_use[i]
    el_name = cap_names[i]
    ax.plot(cum_mass, data[:, el], label = el_name)
    
transx = ax.get_xaxis_transform()

for j in range(len(zones)-1):
    ax.axvline(x = cum_mass[zones[j]], linestyle = '--', c = 'grey', lw = 0.7)
    
    if j < 3:
        pass
    else:
        fig.text(x = cum_mass[zones[j]],
                        y = 0.88, s = zone_names[j], transform = transx)

ax.axhline(y = 1, linestyle = '--', c = 'black')
    
ax.set_xlabel('Ejecta Mass Coordinate [$M_{\odot}$]')
ax.set_ylabel('Mass Fraction')

ax.set_yscale('log')
ax.set_xlim(cum_mass[0], cum_mass[-1])
ax.set_ylim(10**-6, 10**1)
ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4])

plt.figlegend(bbox_to_anchor=(0.03, 1.00, 1, 0.01), loc= 'center',
              borderaxespad = 0, handletextpad = 0.3, ncol=9 ,columnspacing=1.0, frameon= False)


fig.tight_layout()

plt.savefig('Figure_2.pdf', dpi = 300, bbox_inches="tight")
plt.show()